In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob

#Data preprocessing to removes @usernames,urls,symbols and makes all text lowercase
def preprocess_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    text = re.sub('@[^\s]+','', text)
    text = text.lower().replace("ё", "е")
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+',' ', text)
    text = re.sub(' +',' ', text)
    return text.strip()

In [2]:
data = pd.read_json("test.jsonl",lines="True")
data.head()


,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
0,NaN,NaN,2020-07-01 16:44:49,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,RT @readyletsgo27: Business Group Complains Tr...,NaN,...,NaN,NaN,NaN,NaN,292,False,{'created_at': 'Wed Jul 01 13:32:31 +0000 2020...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,"{'id': 891350796972969985, 'id_str': '89135079..."
1,NaN,NaN,2020-07-01 16:44:48,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,"RT @AndrewBatesNC: ""Who's the absent candidate...",NaN,...,NaN,NaN,NaN,NaN,341,False,{'created_at': 'Wed Jul 01 12:53:08 +0000 2020...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,"{'id': 1206974012095639558, 'id_str': '1206974..."
2,NaN,NaN,2020-07-01 16:44:48,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,RT @amvetsupport: That moron trump vows to vet...,NaN,...,NaN,NaN,NaN,NaN,831,False,{'created_at': 'Wed Jul 01 15:16:16 +0000 2020...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,"{'id': 1043252638656147456, 'id_str': '1043252..."
3,NaN,NaN,2020-07-01 16:44:48,"[72, 125]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,1,False,@ClareTyne @mesainy @HKrassenstein @realDonald...,NaN,...,NaN,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 1223446325758394369, 'id_str': '1223446..."
4,NaN,NaN,2020-07-01 16:44:49,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,RT @marklevinshow: 1. Funny how Biden barely c...,NaN,...,NaN,NaN,NaN,NaN,7863,False,{'created_at': 'Wed Jul 01 12:24:39 +0000 2020...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,"{'id': 3266646908, 'id_str': '3266646908', 'na..."


In [3]:
#load data from csv/excel/json
#data=pd.read_excel('tweets.xlsx') 
#data.head()

In [4]:
text = [preprocess_text(t) for t in data.iloc[:]['full_text']]

#storing preprocessed texts in a separate column 
data["processed_text"]= text

#example
print(data.iloc[3]['full_text'])
print(data.iloc[3]["processed_text"])

@ClareTyne @mesainy @HKrassenstein @realDonaldTrump @NYCMayor @JoeBiden Yep.  Torturing that guy because the Dems don’t care.
yep torturing that guy because the dems don t care


In [5]:
#data.head(2)

In [6]:
#Tokenizing 
tokens = [nltk.word_tokenize(t) for t in data.iloc[:]['processed_text']]
#print(tokens)

In [7]:
#Textblob

#calling the Textblob function for each text
blobs = [TextBlob(t) for t in data.iloc[:]["processed_text"]]

selected = data[["id_str","full_text","processed_text"]]
df=selected.copy()

#getting the polarity and subjectivity of the blob
polarity=[]
for i in range(data.shape[0]):
    polarity.append(blobs[i].sentiment)
    
df["Polarity/subjectivity"]= polarity

sentiment= []
for j in range(data.shape[0]):
    if polarity[j][0]>0:
        sentiment.append("Positive")
    elif polarity[j][0]<0:
        sentiment.append("Negative")
    else:
        sentiment.append("Neutral")
        
df["Sentiment"]= sentiment
pd.options.display.max_colwidth = 150
df[["processed_text","Polarity/subjectivity","Sentiment"]]

,processed_text,Polarity/subjectivity,Sentiment
0,rt business group complains trump h 1b reform boosting u s graduates big tech is whining that they will have to hire amer,"(0.0, 0.1)",Neutral
1,rt who s the absent candidate now sleepy joe is signaling that he s very much awake and dialed into a moment where,"(0.26, 0.26)",Positive
2,rt that moron trump vows to veto the defense bill if it includes renaming bases so once again military salaries and defens,"(-0.45, 0.55)",Negative
3,yep torturing that guy because the dems don t care,"(0.0, 0.0)",Neutral
4,rt 1 funny how biden barely criticizes putin and on the china virus doesn t criticize xi instead he smears trump the,"(0.15, 0.55)",Positive
5,arizona reports record spike in new coronavirus cases amp deaths ahead of pence s visit his megachurch stunt took place 5 days ago in phoenix url,"(0.13636363636363635, 0.45454545454545453)",Positive
6,debate president trump prove you don t have dementia dementiajoecantdebate joebidenscaredtodebate,"(0.0, 0.0)",Neutral
7,you disrespected our soldiers in afganistan so your not and better,"(0.5, 0.5)",Positive
8,rt breaking the taliban has just admitted to accepting money from russia to kill u s troops this is not a hoax vla,"(0.0, 0.0)",Neutral
9,rt breaking the taliban has just admitted to accepting money from russia to kill u s troops this is not a hoax vla,"(0.0, 0.0)",Neutral
